In [1]:
import csv,datetime,time
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import matplotlib.dates as mdate
import psycopg2
import sys

import pandas as pd

import plotting_functions as plt2


In [2]:
dsn  = "dbname=gm2_online_prod user=gm2_reader host=g2db-priv port=5433"
conn = psycopg2.connect(dsn)
curr = conn.cursor()


In [22]:
## choose data set (low kick)

# start_run = 16669
# end_run = 16714

## high kick

# start_run = 16110
# end_run = 16256

## (9day)
# start_run = 16355
# end_run = 16539

## endgame
start_run = 16908
end_run = 17528

In [23]:
# get times by subrun

subrun_time_columns = ['run', 'subrun', 'start_time', 'end_time', 'start_gps', 'end_gps']

sql = "select "+", ".join(subrun_time_columns)+" from gm2dq.subrun_time where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)

curr.execute(sql)
conn.commit()
subrun_time = curr.fetchall()

subrun_time_df = pd.DataFrame(subrun_time, columns=subrun_time_columns)

In [24]:
print subrun_time_df.shape
subrun_time_df.head()

(139201, 6)


,run,subrun,start_time,end_time,start_gps,end_gps
0,16908,0,2018-06-06 18:25:40,2018-06-06 18:26:22,2018-06-06 18:26:07.031910,2018-06-06 18:26:21.061910
1,16908,1,2018-06-06 18:26:23,2018-06-06 18:26:37,2018-06-06 18:26:21.081910,2018-06-06 18:26:35.101920
2,16908,2,2018-06-06 18:26:38,2018-06-06 18:26:51,2018-06-06 18:26:35.731920,2018-06-06 18:26:49.771930
3,16908,3,2018-06-06 18:26:52,2018-06-06 18:27:06,2018-06-06 18:26:49.781930,2018-06-06 18:27:04.441940
4,16908,4,2018-06-06 18:27:07,2018-06-06 18:27:20,2018-06-06 18:27:04.451940,2018-06-06 18:27:18.491950


In [25]:
# get ctags by subruns

ctagswithdqc_columns = ['run', 'subrun', 'ctags', 't0val', 'fills']

sql = "select "+", ".join(ctagswithdqc_columns)+" from gm2dq.ctagswithdqc where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)

curr.execute(sql)
conn.commit()
ctagswithdqc = curr.fetchall()

ctagswithdqc_df = pd.DataFrame(ctagswithdqc, columns=ctagswithdqc_columns)

In [26]:
print ctagswithdqc_df.shape
ctagswithdqc_df.head()

(136377, 5)


,run,subrun,ctags,t0val,fills
0,16908,0,0,0.0,0
1,16908,1,0,0.0,0
2,16908,2,0,0.0,0
3,16908,3,0,0.0,0
4,16908,4,0,0.0,0


In [27]:
# get subrun status database

subrun_status_columns = ['run', 'subrun', 'quad_condition', 'kicker_condition', 'quad_ok',
                         'ctags_ok', 'losses_ok', 'fillcuts_ok', 'field_ok', 'trolley_period', 'field_period',
                         'ctags_loose_ok', 'quad_loose_ok', 'ctags_repeat_ok', 'losses_repeat_ok', 'fillcuts_repeat_ok']

sql = "select "+", ".join(subrun_status_columns)+" from gm2dq.subrun_status where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)

curr.execute(sql)
conn.commit()
subrun_status = curr.fetchall()

subrun_status_df = pd.DataFrame(subrun_status, columns=subrun_status_columns)

In [28]:
print subrun_status_df.shape
subrun_status_df.head()

(139201, 16)


,run,subrun,quad_condition,kicker_condition,quad_ok,ctags_ok,losses_ok,fillcuts_ok,field_ok,trolley_period,field_period,ctags_loose_ok,quad_loose_ok,ctags_repeat_ok,losses_repeat_ok,fillcuts_repeat_ok
0,16908,0,None,None,False,False,False,False,True,1.0,4.0,None,None,None,None,None
1,16908,1,None,None,False,False,False,False,True,1.0,4.0,None,None,None,None,None
2,16908,2,None,None,False,False,False,False,True,1.0,4.0,None,None,None,None,None
3,16908,3,None,None,False,False,False,False,True,1.0,4.0,None,None,None,None,None
4,16908,4,None,None,False,False,False,False,True,1.0,4.0,None,None,None,None,None


In [29]:
# merge times, ctags, status into one subrun dataframe

subrun_df = pd.merge(subrun_time_df, ctagswithdqc_df)
subrun_df = pd.merge(subrun_df, subrun_status_df)
subrun_df['ok'] = subrun_df['quad_ok'] & subrun_df['ctags_ok'] & subrun_df['losses_ok'] & subrun_df['fillcuts_ok'] & subrun_df['field_ok']

subrun_df['start_time'] = subrun_df['start_time'].astype(np.int64)/1e9 + 5*60*60
subrun_df['end_time'] = subrun_df['end_time'].astype(np.int64)/1e9 + 5*60*60
subrun_df['start_gps'] = subrun_df['start_gps'].astype(np.int64)/1.0e9 + 5*60*60
subrun_df['end_gps'] = subrun_df['end_gps'].astype(np.int64)/1.0e9 + 5*60*60

# subrun_df.to_hdf('60hr_subrun.h5', key='ctag')

print subrun_df.shape

(136377, 24)


In [ ]:
print type(subrun_df['kicker_condition'].iloc[100].decode('utf-8'))

In [ ]:
subrun_df.dtypes

In [ ]:
print subrun_df['start_gps'].iloc[16]
print subrun_df['start_time'].iloc[16]

In [ ]:
subrun_df[subrun_df['ok']==True].head()

In [ ]:
fig, ax = plt.subplots()
ax.plot(times,ctags,marker="o",linewidth=0,color="green",markeredgewidth=0)
date_fmt = '%d-%b\n%H:%M'
date_formatter = mdate.DateFormatter(date_fmt)
hours = mdate.HourLocator(interval = 12)
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(date_formatter)
ax.set_ylabel("# CTAGs")
ax.set_title("CTags in 60Hr dataset")
fig.autofmt_xdate()
# plt.savefig("ctag_60Hr.png")

In [ ]:
# get database entries for ctags from DQM

start_time = "2018-04-22 00:00:00"
end_time   = "2018-04-22 23:59:59"

sql = "select * from gm2ctag_dqm where time >= '%s' and time <= '%s' order by time DESC" % (start_time, end_time)

curr.execute(sql)
conn.commit()
ctag_dqm = curr.fetchall()

ctag_dqm = np.array(ctag_dqm)

for row in ctag_dqm:
    row[0] = time.mktime(row[0].timetuple())
    
ctag_dqm = ctag_dqm[ctag_dqm[:,0].argsort()]

# plt.plot(ctag_dqm[:,0], ctag_dqm[:,1], '.')